# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [1]:
import pandas as pd
import datetime

---
## 2. Load data

In [2]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [ ]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


In [ ]:
df.columns

Index(['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_p

---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [4]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [5]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

In [6]:
df['month_activated'] = df['date_activ'].dt.month
df['year_activated'] = df['date_activ'].dt.year


In [7]:
df = pd.get_dummies(df, columns=['origin_up', 'has_gas'])


In [8]:
df['cons_deviation_ratio'] = df['cons_12m'] / df['forecast_cons_12m']


In [9]:
correlation = df.corr()
features_to_remove = [col for col in correlation.columns if correlation[col].abs().any() > 0.8] # Threshold for high correlation

df.drop(features_to_remove, axis=1, inplace=True)


<ipython-input-9-ee68bccdaf71>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.corr()


In [10]:
# Filter for December and preceding January data
december_data = price_df[price_df['price_date'].dt.month == 12]
preceding_january_data = price_df[price_df['price_date'].dt.month == 1]

# Ensure consistent year alignment
preceding_january_data = preceding_january_data[preceding_january_data['price_date'].dt.year == december_data['price_date'].min().year - 1]

# Calculate the difference in off-peak prices
price_df['diff_off_peak_dec_jan'] = december_data['price_off_peak_var'] - preceding_january_data['price_off_peak_var']


In [11]:
price_df['off_peak_price_rolling_mean'] = price_df['price_off_peak_var'].rolling(window=7).mean()


In [12]:
price_df['off_peak_price_std'] = price_df['price_off_peak_var'].rolling(window=14).std()


In [ ]:
price_df.columns

Index(['id', 'price_date', 'price_off_peak_var', 'price_peak_var',
       'price_mid_peak_var', 'price_off_peak_fix', 'price_peak_fix',
       'price_mid_peak_fix', 'diff_off_peak_dec_jan',
       'off_peak_price_rolling_mean', 'off_peak_price_std'],
      dtype='object')

In [13]:
if 'cons_12m' in df.columns:
    # Column exists, proceed with the code
    bins = [0, 500, 1000, df['cons_12m'].max()]  # Example bins for consumption
    labels = ['Low', 'Medium', 'High']
    df['cons_bin'] = pd.cut(df['cons_12m'], bins=bins, labels=labels)
else:
    # Column does not exist, handle the error
    print("Error: Column 'cons_12m' not found in DataFrame.")




Error: Column 'cons_12m' not found in DataFrame.


In [14]:
df.columns

Index(['id', 'channel_sales', 'date_activ', 'date_end', 'date_modif_prod',
       'date_renewal'],
      dtype='object')

In [15]:
df['churned'] = df['date_end'].notnull().astype(int)  # 1 for churned, 0 otherwise



In [16]:
df['activation_month'] = df['date_activ'].dt.month
df['activation_year'] = df['date_activ'].dt.year


In [20]:
import numpy

In [23]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score





In [24]:
# Assuming features other than 'churned' and 'tenure' are in X
X = df.drop(['churned'], axis=1)
y = df['churned']


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [28]:
X_train.columns

Index(['id', 'channel_sales', 'date_activ', 'date_end', 'date_modif_prod',
       'date_renewal', 'activation_month', 'activation_year', 'tenure_years'],
      dtype='object')

In [29]:
X_train = X_train.apply(pd.to_numeric, errors='coerce')  # Attempt conversion, handle errors


In [31]:
X_train.head()

,id,channel_sales,date_activ,date_end,date_modif_prod,date_renewal,activation_month,activation_year,tenure_years
740,NaN,NaN,1293062400000000000,1482451200000000000,1446336000000000000,1451088000000000000,12,2010,14
3468,NaN,NaN,1357344000000000000,1483574400000000000,1432425600000000000,1452038400000000000,1,2013,11
14013,NaN,NaN,1287446400000000000,1476835200000000000,1440633600000000000,1445299200000000000,10,2010,14
1188,NaN,NaN,1365638400000000000,1460332800000000000,1365638400000000000,1428796800000000000,4,2013,11
13098,NaN,NaN,1295913600000000000,1485302400000000000,1450224000000000000,1453766400000000000,1,2011,13


In [33]:
X_train = X_train.drop('id', axis=1)  # Drop the 'id' column before splitting


In [34]:
X_train = X_train.drop('channel_sales', axis=1)  # Drop the 'id' column before splitting


In [35]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [37]:
X_test.columns

Index(['id', 'channel_sales', 'date_activ', 'date_end', 'date_modif_prod',
       'date_renewal', 'activation_month', 'activation_year', 'tenure_years'],
      dtype='object')

In [38]:
X_test = X_test.drop('id', axis=1)

In [40]:
X_test = X_test.drop('channel_sales', axis=1)

In [42]:
X_test= X_test.apply(pd.to_numeric, errors='coerce')

In [43]:
y_pred = model.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
